In [1]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from collections import Counter
import heapq

# Load SpaCy English model
nlp = spacy.load('en_core_web_sm')

In [2]:
# Sample legal document text (you would replace this with your actual document loading code)
document_text = """
This Agreement (the "Agreement") is entered into as of [Date], by and between [Party A], with a principal place of business at [Address] ("Party A"), and [Party B], with a principal place of business at [Address] ("Party B").

WHEREAS, Party A and Party B desire to enter into this Agreement to govern their business relationship;

NOW, THEREFORE, in consideration of the mutual covenants and promises contained herein, the parties hereto agree as follows:

1. Definitions.
   (a) "Confidential Information" means any non-public information disclosed by one party to the other party, which is designated as confidential or which reasonably should be understood to be confidential.
   (b) "Term" means the duration of this Agreement as specified in Section 8 below.

2. Confidentiality Obligations.
   (a) Party B agrees to keep confidential all Confidential Information disclosed by Party A.
   (b) Confidential Information shall not include information that: (i) is or becomes publicly known through no breach of this Agreement by the receiving party; (ii) is rightfully received from a third party without breach of any obligation of confidentiality; (iii) is independently developed by the receiving party without reference to or use of the disclosing party's Confidential Information.

3. Term and Termination.
   (a) This Agreement shall commence on the Effective Date and continue in effect until terminated as provided herein.
   (b) Either party may terminate this Agreement upon [X] days' written notice to the other party.

4. Miscellaneous.
   (a) This Agreement constitutes the entire agreement between the parties with respect to the subject matter hereof and supersedes all prior agreements and understandings, oral and written, between the parties.

IN WITNESS WHEREOF, the parties hereto have executed this Agreement as of the date first above written.
"""


In [3]:
# Function to preprocess and tokenize text
def preprocess_text(text):
    # Tokenize the text
    doc = nlp(text.lower())
    tokens = [token.text for token in doc if not token.is_stop and not token.is_punct]
    return tokens

In [4]:
# Function to perform extractive summarization
def extractive_summarization(text, num_sentences=3):
    # Tokenize and preprocess the text
    tokens = preprocess_text(text)
    
    # Calculate word frequency
    word_freq = Counter(tokens)
    max_freq = max(word_freq.values())
    for word in word_freq.keys():
        word_freq[word] = (word_freq[word] / max_freq)
    
    # Calculate sentence scores based on word frequency
    sentence_scores = {}
    sentences = text.split('.')
    for sentence in sentences:
        for word in preprocess_text(sentence):
            if word in word_freq.keys():
                if len(sentence.split(' ')) < 30:  # Consider only sentences with fewer than 30 words
                    if sentence not in sentence_scores.keys():
                        sentence_scores[sentence] = word_freq[word]
                    else:
                        sentence_scores[sentence] += word_freq[word]
    
    # Get top sentences based on scores
    summary_sentences = heapq.nlargest(num_sentences, sentence_scores, key=sentence_scores.get)
    summary = ' '.join(summary_sentences)
    return summary

In [5]:
# Example usage
if __name__ == '__main__':
    summarized_text = extractive_summarization(document_text)
    print("Original Document:\n", document_text)
    print("\nSummarized Document:\n", summarized_text)

Original Document:
 
This Agreement (the "Agreement") is entered into as of [Date], by and between [Party A], with a principal place of business at [Address] ("Party A"), and [Party B], with a principal place of business at [Address] ("Party B").

WHEREAS, Party A and Party B desire to enter into this Agreement to govern their business relationship;

NOW, THEREFORE, in consideration of the mutual covenants and promises contained herein, the parties hereto agree as follows:

1. Definitions.
   (a) "Confidential Information" means any non-public information disclosed by one party to the other party, which is designated as confidential or which reasonably should be understood to be confidential.
   (b) "Term" means the duration of this Agreement as specified in Section 8 below.

2. Confidentiality Obligations.
   (a) Party B agrees to keep confidential all Confidential Information disclosed by Party A.
   (b) Confidential Information shall not include information that: (i) is or becomes p